# IMPORTS

In [184]:
import os
import torch
import torchaudio

In [185]:
import sounddevice as sd

In [186]:
os.listdir('./save/state')

['ss_00.pt',
 'ss_01.pt',
 'ss_02.pt',
 'ss_03.pt',
 'ss_04.pt',
 'ss_05.pt',
 'ss_06.pt',
 'ss_07.pt',
 'ss_08.pt',
 'ss_09.pt']

In [187]:
from importlib.machinery import SourceFileLoader
mi_clasificador = SourceFileLoader('clasificador_padre', '../libs/clasificador_padreV2.py').load_module()
folder_path = "./save/state/"


In [188]:
version = os.listdir(folder_path)[-1]
model_path = os.path.join(folder_path, version)

In [189]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

CARGO EL MODELO

In [190]:
#Metaparameters for MFCC transformer
#parametros para los calculos del mell
TARGET_SR = 16000  # Normalmente el audio se sule usar a 16k aunque encontre papers que trabajan a 22050 o a 22k (ver V2)
N_FFT = 1024  #muestras de la fft
W_LEN = 800  # Numero de muestras para la ventan de la  fft (seg_de_ventan *sr) 
H_LEN = 320 # paso de la ventana entre una fft y la siguiente (paso * sr)
N_MELS = 40#26
N_MFCC = 32#13
MFCCCalculator = torchaudio.transforms.MFCC(sample_rate = TARGET_SR,
                                            n_mfcc = int(N_MFCC),
                                            dct_type = 2,
                                            norm = 'ortho',
                                            log_mels = False,
                                            melkwargs = 
                                            {
                                                "n_fft": N_FFT,          # Size of FFT (2048)
                                                "win_length": W_LEN,     # Actual window size (400 samples = 25ms)
                                                "hop_length": H_LEN,     # Hop length (160 samples = 10ms)
                                                "n_mels": N_MELS,        # Number of Mel bins (40)
                                                "center": False
                                                },)

In [191]:
def log_weight_norms(model, epoch):
    total_l2_norm_sq = 0.0
    print(f"\nEpoch {epoch+1} - Weight L2 Norms:")
    for name, param in model.named_parameters():
        if param.requires_grad and "weight" in name: # Comúnmente los pesos se llaman 'weight'
            norm = torch.norm(param.data, p=2) # Calcula la norma L2
            #print(f"  Layer '{name}': {norm.item():.4f}")
            total_l2_norm_sq += norm.item()**2
        # Podrías añadir un chequeo para 'bias' si también te interesan

    overall_norm = total_l2_norm_sq**0.5
    print(f"  Overall Network L2 Norm (Weights Only): {overall_norm:.4f}")
    # Puedes guardar este valor en una lista para graficarlo después
    return overall_norm

In [193]:
clasificador = mi_clasificador.onlyWoman_MFCC_16k_v8(['Ariana Grande', 'Katy Perry', 'Taylor Swift'], MFCCCalculator).to(device)
clasificador.load_state_dict(torch.load(model_path))
clasificador.eval()

onlyWoman_MFCC_16k_v8(
  (transformer): MFCC(
    (amplitude_to_DB): AmplitudeToDB()
    (MelSpectrogram): MelSpectrogram(
      (spectrogram): Spectrogram()
      (mel_scale): MelScale()
    )
  )
  (inPut): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (chanelUp): Sequential(
    (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (justTime): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1,

#PARAMETROS PARA PROCESAR EL AUDIO

In [194]:
dic = {0:"Ari", 1:"Katy", 2:"Taylor"}

In [204]:
record = sd.rec(int(DUR_SAMPLE*TARGET_SR), samplerate=TARGET_SR, channels=CHANNELS, dtype='float32', blocking= True)
record_tensor = torch.from_numpy(record.T).to(device)
#record_tensor = record_tensor / record_tensor.abs().max()
record_tensor= clasificador.audio_norm(record_tensor)
#mfcc = MFCCCalculator(record_tensor)
clasificador.eval()
res = clasificador(record_tensor)
pred = dic[int(torch.argmax(res))]
print(pred)
print(res)

Ari
tensor([[0.8294, 0.0686, 0.1020]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [205]:
res

tensor([[0.8294, 0.0686, 0.1020]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

# eval de pesos

In [85]:
for i in range(len(os.listdir(folder_path))):
    patito = os.path.join(folder_path, os.listdir(folder_path)[i])
    clasificador.load_state_dict(torch.load(patito))
    log_weight_norms(clasificador, i)


Epoch 1 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 52.2903

Epoch 2 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 49.1771

Epoch 3 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 47.9840

Epoch 4 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 47.1420

Epoch 5 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 46.4043

Epoch 6 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 45.7046

Epoch 7 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 45.0288

Epoch 8 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 44.3641

Epoch 9 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 43.7156

Epoch 10 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 43.0746

Epoch 11 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 42.4398

Epoch 12 - Weight L2 Norms:
  Overall Network L2 Norm (Weights Only): 41.8093

Epoch 13 - Weight L2 Norms:
  Overall Network L2 Norm (Weigh